Import useful modules 

In [1]:
import sys
import os
### discomment above two line ones to add the feelpp path with nirb module in the python environement
# feelpath = os.path.join('/data/home/elarif/devel/feelpp/build/python/install/lib/python3/dist-packages')
# sys.path.append(feelpath)
from feelpp.mor.nirb.nirb import *
from feelpp.mor.nirb.utils import init_feelpp_environment
from feelpp.mor.nirb.nirb_perf import ComputeErrorSampling, ComputeErrors
import time
import json
import pandas as pd

Choose config file

In [2]:
config_file = os.path.join("/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.cfg")
N = None # number of basis function saved
outdir = None

Set feelpp environnement 

In [3]:
cfg = feelpp.readCfg(config_file)
toolboxType = cfg['nirb']['toolboxType'][0]
e = init_feelpp_environment(toolboxType, config_file)

nirb_file = feelpp.Environment.expand(cfg['nirb']['filename'][0])
config_nirb = feelpp.readJson(nirb_file)['nirb']

[ Starting Feel++ ] application feelpp-mor-nirb version 0.1 date 2023-Jan-24
 . feelpp-mor-nirb files are stored in /home/saigre/Documents/DELETE_ME/nirb/heat/np_1
 .. logfiles :/home/saigre/Documents/DELETE_ME/nirb/heat/np_1/logs
Reading /home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.cfg...


Get number of basis functions saved. By defaults, it will take the value filled in the model path. 

In [4]:
nbSnap=N
if nbSnap==None:
    nbSnap = config_nirb['nbSnapshots']

Set the input/output path 

In [5]:
doGreedy = config_nirb['greedy-generation']
doRectification = config_nirb['doRectification']
rectPath = ["noRect", "Rect"][doRectification]
greedyPath = ["noGreedy", "Greedy"][doGreedy]

if outdir is None:
    RESPATH = f"results/{rectPath}/{greedyPath}"
else:
    RESPATH = outdir

Initialize nirb offline class 

In [6]:
nirb_on = nirbOnline(**config_nirb)
nirb_on.initModel()

[NIRB] ToolboxModel created, but the objects have not yet been initialized. Please call initModel() or setModel() to initialize the objects.
[NIRB] Initialization done
[loadMesh] Loading mesh in format geo+msh: "/home/saigre/Documents/code/feelppheat(2,1)
/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.geo"
[loadMesh] Use default geo desc: /home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.geo 0.01 
[NIRB] Initialization done
[NIRB] Number of nodes on the fine mesh : 11860
[CRBmodelProperties] Loading Model Properties : "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.json"
heat(2,1)
[NIRB] Number of nodes on the coarse mesh : 156
[loadMesh] Loading mesh in format geo+msh: "/home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.geo"
[loadMesh] Use default geo desc: /home/saigre/Documents/code/feelpp/python/pyfeelpp-mor/feelpp/mor/nirb/model/square/square.geo 0

Load offline data

In [7]:
err = nirb_on.loadData(path=RESPATH, nbSnap=nbSnap)
assert err == 0, "Error while loading data"

[NIRB] Loading data from /home/saigre/Documents/DELETE_ME/nirb/heat/np_1/results/Rect/Greedy
[NIRB] Data loaded from /home/saigre/Documents/DELETE_ME/nirb/heat/np_1/results/Rect/Greedy
[NIRB] Number of basis functions loaded : 10


Choose the parameter to cumpute nirb approximate solution 

In [8]:
mu = nirb_on.Dmu.min() # get the min value of parameters given in the model
mu.view()

Bi	0.01
k1	0.1
k2	0.1
k3	0.1


Compute approximate solution 

In [9]:
uHh = nirb_on.getOnlineSol(mu)
uh = nirb_on.getToolboxSolution(nirb_on.tbFine, mu)

[NIRB] Solution computed with Rectification post-process 


In [10]:
l2Mat = nirb_on.generateOperators()
print(nirb_on.normMat(l2Mat, uHh-uh))
print((uHh - uh).linftyNorm())

0.04326018909582374
0.18563488118140725


In [11]:
nirb_on.initExporter("ExporterNORECT")
nirb_on.exportField(uHh, "uHh")
nirb_on.exportField(uh, "uh")
nirb_on.saveExporter()

Get solution for different size of subspaces

In [12]:
solutions = []
for n in range(1, nirb_on.N+1):
    solutions.append(nirb_on.getOnlineSol(mu, n))

[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 
[NIRB] Solution computed with Rectification post-process 


In [13]:
nirb_on.initExporter("NirbN")
nirb_on.exportField(uh, "uh")
for i, uHnN in enumerate(solutions):
    nirb_on.exportField(uHnN, f"uHn{i+1:02}")
nirb_on.saveExporter()

Show infos 

In [12]:
if feelpp.Environment.isMasterRank():
    print(f"[NIRB] Online part Done !")

[NIRB] Online part Done !


# NIRB perfs 

We can compare the above nirb solution with the FE one. 

In [13]:
errorN = ComputeErrors(nirb_on, mu, h1=True)
df = pd.DataFrame(errorN)
df['N'] = nirb_on.N

Display errors 

In [14]:
if feelpp.Environment.isMasterRank():
    print(f"[NIRB online] with {nirb_on.N} basis functions ")
    print(f"[NIRB online] Convergence error between nirb and FE solution ")
    print(df.head())

[NIRB online] with 10 basis functions 
[NIRB online] Convergence error between nirb and FE solution 
   l2(uh-uHn)  l2(uh-uHn)rec  l2(uh-uhn)    l2(uh)  l2(uh-uH)  h1(uh-uHn)  \
0    0.006607       0.006658    0.006405  3.905143   0.002682    0.114305   

   h1(uh-uHn)rec  h1(uh-uhn)    h1(uh)  h1(uh-uH)   N  
0       0.113956    0.114576  3.513885   0.137766  10  


Or get error on a sampling of parameters. This will compare the nirb solution with the FE one on the fine mesh computed from parameter in the sampling (take little time). 

In [15]:
Nsample = 30
errorN = ComputeErrorSampling(nirb_on, Nsample=Nsample, h1=True)
df = pd.DataFrame(errorN)
df['N'] = nirb_on.N

[NIRB] ComputeErrorSampling: 100%|██████████████████████████████████████████████████████| 30/30 [00:02<00:00, 10.58it/s]


Display errors 

In [16]:

if feelpp.Environment.isMasterRank():
    print(f"[NIRB online] with {nirb_on.N} basis functions ")
    print(f"[NIRB online] Error on a sampling of {df.shape[0]} parameters ")
    data_mean = df.mean(axis=0)
    print("[NIRB online] Mean of errors ")
    print(data_mean)
    data_min = df.min(axis=0)
    print("[NIRB online] Min of errors ")
    print(data_min)
    data_max = df.max(axis=0)
    print("[NIRB online] Max of errors ")
    print(data_max)

[NIRB online] with 10 basis functions 
[NIRB online] Error on a sampling of 30 parameters 
[NIRB online] Mean of errors 
l2(uh-uHn)        0.005083
l2(uh-uHn)rec     0.004873
l2(uh-uhn)        0.004599
l2(uh)            1.505833
l2(uh-uH)         0.002356
h1(uh-uHn)        0.069181
h1(uh-uHn)rec     0.069215
h1(uh-uhn)        0.068691
h1(uh)            0.687786
h1(uh-uH)         0.074870
N                10.000000
dtype: float64
[NIRB online] Min of errors 
l2(uh-uHn)        0.000718
l2(uh-uHn)rec     0.000419
l2(uh-uhn)        0.000416
l2(uh)            0.489774
l2(uh-uH)         0.000415
h1(uh-uHn)        0.007113
h1(uh-uHn)rec     0.006154
h1(uh-uhn)        0.006162
h1(uh)            0.329100
h1(uh-uH)         0.022834
N                10.000000
dtype: float64
[NIRB online] Max of errors 
l2(uh-uHn)        0.025149
l2(uh-uHn)rec     0.025016
l2(uh-uhn)        0.024925
l2(uh)            7.059206
l2(uh-uH)         0.014794
h1(uh-uHn)        0.353910
h1(uh-uHn)rec     0.354452
h1(uh-uh

Save covergence error values in a `csv` file for making statistics or vizualisation   

In [17]:
file =RESPATH +f"/errors{Nsample}Params.csv"
header = not os.path.isfile(file)
df.to_csv(file, mode='a', index=False, header=header)